In [ ]:
import os
from io import open
import time
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
import sys
import torch.quantization
from torch.quantization import QuantStub, DeQuantStub

In [ ]:
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.utils.misc import Timer
import torch.quantization

In [ ]:
# Post training Quantization : Dynamic

In [ ]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [ ]:
class_names = [name.strip() for name in open('models/voc-model-labels.txt').readlines()]
num_classes = len(class_names)
model = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)
model.load_state_dict(torch.load('models/mb2-ssd-lite-mp-0_686.pth',map_location=torch.device('cpu')))
model.eval()


In [ ]:
model.fuse_model()

In [ ]:
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
print(model.qconfig)

In [ ]:
torch.quantization.prepare(model, inplace=True)
# evaluate(per_channel_quantized_model,criterion, data_loader, num_calibration_batches)
torch.quantization.convertert(model, inplace=True)

In [ ]:
print("Size of model after quantization")
print_size_of_model(model)

In [ ]:
import cv2

In [ ]:
image = cv2.imread("test.jpg")
if image.shape[2] == 1:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
else:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


In [ ]:
image  = cv2.transform(image, boxes, labels)

In [ ]:
predictor = create_mobilenetv2_ssd_lite_predictor(model, candidate_size=200, device='cpu')
boxes, labels, probs = predictor.predict(image)

In [ ]:
torch.save(model.state_dict(), 'new_model.pth')

In [ ]:
model.save('./test.pth') #Save to a dictionary

In [ ]:
model.load('./test.pth') #Reload to the model  OKKKKKKKKKKKKKK!!!

In [ ]:
# If you recreate a empty model  NOT okkkkkkkkkkkkkkkk!!!
model_load = create_mobilenetv2_ssd_lite(len(class_names), is_test=True) 

In [ ]:
#Then you reload you quantized model to this empty file
#This is the problem
# It is not working because the keys are different now
model_load.load('./test.pth')

In [ ]:
img = cv2.imread("readme_ssd_example.jpg")

In [ ]:
predictor = create_mobilenetv2_ssd_lite_predictor(model, candidate_size=200, device='cpu')

In [ ]:
#https://github.com/pytorch/pytorch/issues/28483
#Check out this issue feels like you didn't do the fuse properly
predictor.predict(img)

In [ ]:
def uninplace(model):
    """Sets all `inplace` values to False"""
    if hasattr(model, 'inplace'):
        model.inplace = False
    if not model.children():
        return
    for child in model.children():
        uninplace(child)

def prep_for_fusion(model, parent_name):
    """Fuses all conv+bn+relu, conv+bn, and conv+relu"""
    if not model.children():
        return []
    result = []
    candidate = []
    for name, child in model.named_children():
        new_name = parent_name + '.' + name
        if new_name[0] == '.':
            new_name = new_name[1:]
        if type(child) == torch.nn.Sequential:
            candidate = []
            result.extend(prep_for_fusion(child, new_name))
        else:
            if len(candidate) == 0 and type(child) == torch.nn.Conv2d:
                candidate = [new_name]
            elif len(candidate) == 1 and type(child) == torch.nn.ReLU:
                candidate.append(new_name)
                result.append(candidate)
                candidate = []
            elif len(candidate) == 1 and type(child) == torch.nn.BatchNorm2d:
                candidate.append(new_name)
            elif len(candidate) == 2:
                if type(child) == torch.nn.ReLU:
                    candidate.append(new_name)
                result.append(candidate)
                candidate = []
    return result

model.eval()

# example = torch.rand(1, 3, 224, 224)
# traced_script_module = torch.jit.trace(model, example)
# traced_script_module.save('vgg16_bn_float32.pt')

uninplace(model)  # This is a hack!

modules_to_fuse = prep_for_fusion(model, '')

model.qconfig = torch.quantization.default_qconfig
fused_model = torch.quantization.fuse_modules(model, modules_to_fuse,inplace=True)

torch.quantization.prepare(fused_model, inplace=True)
quantized = torch.quantization.convert(fused_model, inplace=True)

q_example = torch.quantize_per_tensor(example, scale=1e-3, zero_point=128,
                                      dtype=torch.quint8)


# traced_script_module = torch.jit.trace(quantized, q_example, check_trace=False)
# traced_script_module.save('vgg16_bn_qint8.pth')